In [5]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO
from lang_sam import LangSAM
import os

def save_mask(mask_np, filename):
    mask_image = Image.fromarray((mask_np * 255).astype(np.uint8))
    mask_image.save(filename)

def print_bounding_boxes(boxes):
    print("Bounding Boxes:")
    for i, box in enumerate(boxes):
        print(f"Box {i+1}: {box}")

def print_detected_phrases(phrases):
    print("\nDetected Phrases:")
    for i, phrase in enumerate(phrases):
        print(f"Phrase {i+1}: {phrase}")

def print_logits(logits):
    print("\nConfidence:")
    for i, logit in enumerate(logits):
        print(f"Logit {i+1}: {logit}")

def find_coin_masks(image):
    # Suppress warning messages
    warnings.filterwarnings("ignore")
    text_prompt = "coin"
    try:
        model = LangSAM()
        masks, boxes, phrases, logits = model.predict(image, text_prompt)

        if len(masks) == 0:
            print(f"No objects of the '{text_prompt}' prompt detected in the image.")
        else:
            # Convert masks to numpy arrays
            masks_np = [mask.squeeze().cpu().numpy() for mask in masks]
            return masks_np

    except (requests.exceptions.RequestException, IOError) as e:
        print(f"Error: {e}")

# returns y_min, y_max, x_min, x_max
def find_boundary_of_coin(nonzero_indices):
    return nonzero_indices[0,:].min(), nonzero_indices[0,:].max(), nonzero_indices[1,:].min(), nonzero_indices[1,:].max()

def generate_coin_images(image_dir):
    image = Image.open(image_dir).convert("RGB")
    masks = find_coin_masks(image)
    image = np.array(image)
    coins = []
    for index in range(len(masks)):
        mask = np.broadcast_to(np.expand_dims(masks[index],-1), image.shape)
        masked_image = mask * image
        nonzero_indices = np.nonzero(masked_image[:,:,0])
        nonzero_indices = np.array(nonzero_indices)
        y_min, y_max, x_min, x_max = find_boundary_of_coin(nonzero_indices)
        masked_image = masked_image[y_min:y_max,x_min:x_max]
        difference_y = 500 - (y_max - y_min)
        difference_x = 500 - (x_max - x_min)
        if difference_y != 0:
            if difference_y % 2 == 0:
                masked_image = np.pad(masked_image, [(difference_y//2, difference_y//2), (0, 0), (0, 0)])
            else:
                masked_image = np.pad(masked_image, [((difference_y-1)//2, (difference_y-1)//2 + 1), (0, 0), (0, 0)])
        if difference_x != 0:
            if difference_x % 2 == 0:
                masked_image = np.pad(masked_image, [(0, 0), (difference_x//2, difference_x//2), (0, 0)])
            else:
                masked_image = np.pad(masked_image, [(0, 0), ((difference_x-1)//2, (difference_x-1)//2 + 1), (0, 0)])
        coins.append(masked_image)
    return coins


/Users/gamze/anaconda3/envs/data_analysis/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/gamze/anaconda3/envs/data_analysis/lib/python3.9/site-packages/groundingdino/models/GroundingDINO/ms_deform_attn.py:31: UserWarning: Failed to load custom C++ ops. Running on CPU mode Only!
  warnings.warn("Failed to load custom C++ ops. Running on CPU mode Only!")


In [10]:
## find subfolder names in a folder
dataset_dir = "coin_images/"
subfolders = os.listdir(dataset_dir)
if '.DS_Store' in subfolders:
    subfolders.remove('.DS_Store')
subfolders

['10c', '50c', '20c', '5c', '1e', '2e']

In [9]:
## find filenames in a folder
dataset_dir = "coin_images/1e/"
image_names = os.listdir(dataset_dir)
if '.DS_Store' in subfolders:
    image_names.remove('.DS_Store')
image_names

['IMG_4198.JPG', 'IMG_4199.JPG', 'IMG_4200.JPG', 'IMG_4197.JPG']

In [13]:
## concat directoriess
output_dir = "padded_coin_images/"
subfolder = os.path.join(output_dir, subfolders[0], 'img.png')


'padded_coin_images/10c/img.png'

In [6]:
## image save etme
visual = 255 * np.ones((500,500))
image = Image.fromarray((visual).astype(np.uint8))
last_index = find_last_index(os.path.join(output_dir, subfolders[0]))
image_name = f"img_{last_index+1}.png"
subfolder = os.path.join(output_dir, subfolders[0], image_name)
image.save(subfolder)

NameError: name 'output_dir' is not defined

In [3]:
strings = ['img_1.png','img_78.png','img_0.png','img_55.png']
def find_last_index(strings):
    max_index = 0
    for string in strings: 
        index = int(string.replace('img_','').replace('.png',''))
        if index > max_index:
            max_index = index
    return max_index
max_index = find_last_index(strings)
